Logic for generating expressions for h_n(x,y,z), which consists of a set of indicator constraints and a real coefficient.

In [40]:
from sympy import symbols, And, Not, simplify, solve
%run constants.ipynb
%run li_handler.ipynb

In [41]:
def no_loser(a,b,c):
    '''
    Returns a list of predicates indicating no player has lost yet 
    in a given state.
    '''
    return [simplify(i) for i in [a>0, b>0, c>0]]

In [42]:
def parentStates(a,b,c, r=V):
    '''
    Returns all feasible 'parent' states of the given state 
    (i.e. possible stacks in the previous round), 
    under restriction `r`.
    '''
    return [s for s in 
            [(2*a,b-a,c), (2*a,b,c-a),
            (a-b,2*b,c), (a,2*b,c-b),
            (a-c,b,2*c), (a,b-c,2*c)] 
            if feasible(*no_loser(*s), r=r)]

In [43]:
@memoized
def coef(n):
    '''
    Returns the real coefficient of h_n.
    '''
    return Fraction(1, 6**n)

In [44]:
def concat(ls):
    ''' 
    Flattens a 2D list. 
    '''
    return [j for l in ls for j in l]

In [45]:
import csv
from itertools import islice

def read(filename, min_n=1, max_n=5):
    '''
    Reads a csv with entries(n, real coefficient, indicator constraints) 
    for n in [`min_n`, `max_n`].
    Returns the parsed entries as a dictionary of n : (coefficient, indicators).
    '''
    hs = {}
    with open(filename, 'r') as fp:
        reader = islice(csv.reader(fp), min_n-1, max_n)
        for row in reader:
            n, c, rs = row
            # parse entries
            hs[int(n)] = (int(c), sympify(rs))
    return hs

In [46]:
def h_mult(n,a,b,c):
    '''
    Returns the non-redundant multiplicative constraints of h_n(a,b,c).
    '''
    return non_redundant(*no_loser(a,b,c), r=V)

def mult_inds(inds, prereqs, r=V):
    '''
    Indicator multiplication.
    Returns [prereqs * sum(inds)], keeping only non-redundant indicators in `prereqs` given restriction `r`.
    '''
    return [i + [p for p in non_redundant(*prereqs, r=V+i)] for i in inds]

@memoized
def _h(n,a,b,c):
    '''
    Returns the additive indicators of h_n(a,b,c), as a 2D list.
    '''
    # constant coef = (1/6)^n
    # so we only keep track of the indicators to be summed up
    if n == 1:
        return mult_inds([[simplify(a<=b)], [simplify(a<=c)]], h_mult(1,a,b,c))
    # list of inequalities representing the region Rn
#     print('n =',n)
#     for s in parentStates(a,b,c):
#         print(s, _h(n-1, *s))
#     print()
    return mult_inds(concat([_h(n-1, *s) for s in parentStates(a,b,c)]), 
                     h_mult(n,a,b,c))

@memoized
def h(n,a,b,c, regenerate):
    '''
    Returns h_n(a,b,c).
    If `regenerate` is True, re-calculate h_n, as opposed to reading from cache.
    '''
    if regenerate:
        return reduce(_h(n,a,b,c), r=V)
    else:
        if (a,b,c) == flip_state(STATE):
            return read(H_YXZ_CACHE, min_n=n-1, max_n=n)[n]
        else:
            return read(H_XYZ_CACHE, min_n=n-1, max_n=n)[n]

# Evaluate with actual x,y,z

In [24]:
MAX_N = 5
H_XYZ = read(H_XYZ_CACHE, max_n=MAX_N)
# H_YXZ = read(H_YXZ_CACHE, max_n=MAX_N)
H_XXZ = read(H_XXZ_CACHE, max_n=MAX_N)
H_XXZ2 = read(H_XXZ_CACHE+'_xyz', max_n=MAX_N)

In [27]:
def eval_h(n,a,b,c):
    '''
    Uses the general h(n,a,b,c) function to evaluate h_n(a,b,c).
    '''
    const, rs = H_XYZ[n]
    vals = {x:a, y:b, z:c}
    return coef(n) * (const + sum([sub(r, vals) for r in rs]))

In [18]:
# def eval_f(n,a,b,c):
#     '''
#     Uses the general h(n,a,b,c) function to evaluate f_n(a,b,c) - alpha_n.
#     '''
#     tot = 0
#     vals = {x:a, y:b, z:c}
#     for i in range(1,n+1):
#         pos_c, pos_rs = H_XYZ[i]
#         neg_c, neg_rs = H_YXZ[i]
#         tot += coef(i) * (pos_c - neg_c + sum([sub(r, vals) for r in pos_rs]) - sum([sub(r, vals) for r in neg_rs]))
#     return tot - thresh(n)

In [19]:
# def eval_dh(n,a,b,c):
#     '''
#     Uses the general h(n,a,b,c) function to evaluate h_n(a,b,c) - h_n(b,a,c)
#     '''
#     vals = {x:a, y:b, z:c}
#     pos_c, pos_rs = H_XYZ[n]
#     neg_c, neg_rs = H_YXZ[n]
#     return coef(n) * (pos_c - neg_c + sum([sub(r, vals) for r in pos_rs]) - sum([sub(r, vals) for r in neg_rs]))

In [20]:
# bound = eval_f(3,4,5,6) + eval_dh(4,7,8,23) + thresh(4)
# print(bound)
# print(float(bound))
# print(bound + thresh(4))  # lower bound on f_4(x,y,z) - alpha_4

In [47]:
def eval_d_dash(n,a,b,c):
    '''
    Evaluates delta'(a,b,c).
    '''
    tot = 0
    vals = {x:a, y:b, z:c}
    for i in range(1,n+1):
        pos_c, pos_rs = H_XYZ[i]
        neg_c, neg_rs = H_XXZ2[i]
        tot += coef(i) * (pos_c - neg_c + sum([sub(r, vals) for r in pos_rs]) - sum([sub(r, vals) for r in neg_rs]))
    return tot - thresh(n)